# XML Parser

**THIS IS ONLY FOR 15-688 STUDENTS**

In this problem, you're going to write a function that will parse (a simplified version of) XML files into a Python object. Although it's not recommended that you use the parser you construct for anything serious (many excellent Python libraries already exist for parsing XML, such as the lxml library), XML files represent a fairly complex file format, that necessitates using regular expressions and recursion (or a stack) to parse these in a reasonably efficient manner.  So while it's not likely that you will need to write you own XML parser, chances are if/when you _do_ need to write a parser for some format for which there exists no good Python library, the techniques you use here will be useful for writing this parser as well.

XML stands for the eXtensible Markup Language.  It appeared as a successor to SGML (standard generalized markup language) and HTML (hypertext markup language, the standard for displaying web pages), but with some additional structure that makes the documents more well-defined; for instance, in HTML it's common for open tags to appear without a corresponding close tag, which is not allowed in pure XML.

You may already be familiar with XML, but if not the official resource for learning about the format is here https://www.w3.org/XML/ and a good resource with some concrete examples is here http://www.w3schools.com/xml/.  We'll assume here that you're broadly familiar with the basic ideas behind XML, and just describe what you need to know to complete the parser for this assignment.

Here is an example XML document:
    
    <?xml version="1.0" encoding="UTF-8"?>
    <!-- This is a comment -->
    <note date="8/31/12">
        <to>Tove</to>
        <from>Jani</from>
        <heading type="Reminder"/>
        <body>Don't forget me this weekend!</body>
        <!-- This is a multiline comment,
             which take a bit of care to parse -->
    </note>
    
There are a few elements here of importance.
1. Tags are denoted `<tag_name>content</tag_name>` where `<tag_name>` is the opening tag and `</tag_name>` is the closing tag.  All text (including whitespace) between these tags is the content.
2. Attributes follow a tag, and are written as a list of `attr_name="attribute_value"` pairs, where we can use either double quotes or single quotes around the attribute value.  If you use double quotes then a single quote can appear in the text and vice versa.  There can be whitespace around the equals sign or not.
3. If a tag has no content `<tag_name attr_name="attribute_value"></tag_name>` can be abbreviated as the open/close tag `<tag_name attr_name="attribute_value"/>`. In some cases, such as in HTML5, you might come across tags that have no content but aren't closed, such as `<meta ...>` and `<link ...>` tags. **However, as we are dealing with XML (a stricter context), tags are required to be closed.** If you're interested, [this document](https://www.w3schools.com/html/html_xhtml.asp) on XHTML vs HTML touches upon this idea.
4. A XML prologue is written as `<?tag_name attr_name="attribute_value"?>`.  It has no close tag.  We'll also consider documents that allow for an HTML declaration, such as `<!DOCTYPE html>` (this will let us parse some HTML documents that are well-formed enough to also parse as valid XML).
5. Comments are denoted by `<!-- comment_text -->`  and the comment text can span multiple lines.

## Q1: Regular expression for identifying tags

First, we'll use regular expressions to identify tags and other elements of XML files.  Specifically, you'll need to create 6 regular expressions that locate open tags, close tags, open/close tags, comments, xml_prolog, and html declarations.  For the open, close and open/close tags, make sure that your regular expression also matches and returns 1) the tag name, and 2) all the attributes.  Note that in our implementation, we actually have the open tag _also_ match open/close tags, but you are free to do this either way (they can match or not).  Comments may be split across multiple lines, but you can assume that all other tags must occur on a single line (without newlines within the tag itself).

In [199]:
import re
import requests
from testing.testing import test

In [215]:
tag_open = re.compile(r'<([^?!/]\w*)(\s*.?/)>') 
tag_close = re.compile(r'<\/([^\!\?]\w*)')
tag_open_close = re.compile(r'<(.*?)(\s*[\w=\"]*)/>')

comment = re.compile("<!--(.*?)-->", re.DOTALL)
xml_prolog = re.compile("<\?(.*)\?>")
html_declaration = re.compile("<!(\w*\s*\w*)\>")

test_snippet = """<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE xml> <!-- not actually valid xml-->
<!-- This is a comment -->
<note date="8/31/12">
    <to>Tove</to>
    <from>Jani</from>
    <heading type="Reminder"/>
    <body><h1>Don't</h1><p>forget me this weekend!</p></body>
    <!-- This is a multiline comment,
         which take a bit of care to parse -->
    Here's some trailing text.
</note>
"""

course_webpage = requests.get("http://www.datasciencecourse.org/").content.decode()

def get_tags_test(get_tags):
    # Test snippet:
    test_val = get_tags(test_snippet)
    test.equal(test_val['tag_open'], [('note', ' date="8/31/12"'), ('to', ''), ('from', ''), ('heading', ' type="Reminder"/'), ('body', ''), ('h1', ''), ('p', '')])
    test.equal(test_val['tag_close'], ['to', 'from', 'h1', 'p', 'body', 'note'])
    test.equal(test_val['tag_open_close'], [('heading', ' type="Reminder"')])
    test.equal(test_val['comment'], [' not actually valid xml', ' This is a comment ', ' This is a multiline comment,\n         which take a bit of care to parse '])
    test.equal(test_val['xml_prolog'], ['xml version="1.0" encoding="UTF-8"'])
    test.equal(test_val['html_declaration'], ['DOCTYPE xml'])

    # Test course webpage:
    test_val = get_tags(course_webpage)
    test.equal(len(test_val['tag_open']), 126)
    test.equal(len(test_val['tag_close']), 102)
    test.equal(len(test_val['tag_open_close']), 24)
    test.equal(len(test_val['comment']), 4)
    test.equal(len(test_val['xml_prolog']), 0)
    test.equal(len(test_val['html_declaration']), 1)

In [216]:
tag_open.findall(test_snippet)

[]

In [217]:
@test
def get_tags(inp):
    return {
        "tag_open": tag_open.findall(inp),
        "tag_close": tag_close.findall(inp),
        "tag_open_close": tag_open_close.findall(inp),
        "comment": comment.findall(inp),
        "xml_prolog": xml_prolog.findall(inp),
        "html_declaration": html_declaration.findall(inp)
    }

### TESTING get_tags: PASSED 10/12
# 0	: Failed: [] is not equal to [('note', ' date="8/31/12"'), ('to', ''), ('from', ''), ('heading', ' type="Reminder"/'), ('body', ''), ('h1', ''), ('p', '')]
# 6	: Failed: 0 is not equal to 126
###



(Note that although there is only one html declaration in the course webpage, there is a field _within_ a comment that looks suspiciously like a declaration, so you can pick up both of these).

## Q2: XML Parser

Using the regular expressions above, now you'll write an XML parser (although technically you don't _have_ to use them, you could try to write a complete XML parser using a single regular extended expression if you really want to, but we would highly advise against this).  Specifically, you should fill in the `__init__` function for the class prototype below.

We've provided you with some test cases to make sure that your parser works correctly.

In [213]:
class XMLNode:
    @staticmethod
    def fromString(s):
        return XMLNode("", {}, s)
    
    attr_double_quote =  re.compile('\s*([^=]+)\s*=\s*"([^"]*)"\s*')
    attr_single_quote =  re.compile("\s*([^=]+)\s*=\s*'([^']*)'\s*")

    def __eq__(self, o):
        if (self.tag != o.tag) or (self.attributes != o.attributes):
            return False
        return all(a == b for a, b in zip(self.children, o.children))

    def __getitem__(self, i):
        return self.children[i]
    
    def __str__(self):
        rv = "".join(str(c) for c in self.children)
        if self.tag:
            attribute_string = (" " + " ".join(f'{k}=\"{v}\"' for k, v in self.attributes.items())).rstrip()
            if self.children:
                return f"<{self.tag}{attribute_string}>{rv}</{self.tag}>"
            return f"<{self.tag}{attribute_string} />"
        return rv

    def __repr__(self):
        return self.__str__()

    def count_tags(self):
        return (1 if self.tag else 0) + sum(c.count_tags() if isinstance(c, XMLNode) else 0 for c in self.children)

    def create_child(self, *a, **kw):
        return XMLNode(*a, **kw)
    
    def __init__(self, tag, attributes, content):
        self.tag = tag # The tag <tag>
        self.attributes = attributes # A dictionary from attributes to values
        self.children = [] # A list of either XMLNode objects and strings, corresponding to tags and text between them.
        self.content = content # A string of everything in the original document inside these tags
        
        position=content.find('<')     #Initial position
        start=re.compile(r'<').search(content,position) 
        
        while start:
            #Check for which content and positions match
            match_O = tag_open.match(content, position)
            match_OCL = tag_open_close.match(content, position)
            match_CL = tag_close.match(content, position)
            match_xml = xml_prolog.match(content, position)
            match_html = html_declaration.match(content, position)
            match_comment = comment.match(content, position)

            #To skip to positions after prologue, declaration and comment:
            if match_xml:
                position = match_xml.end()
            elif match_html:
                position = match_html.end()
            elif match_comment:
                position = match_comment.end()
            elif match_O:
                attr = match_O.group(2).strip('/')
                update_dict={}
                attr_check = re.findall(r'(\w+="[^"]*)"', attr)
                if attr_check == []:
                    update_dict ={}
                else:
                    #get the attribute in the required format
                    for i in range(len(attr_check)):         #in case of more attributes
                        date = attr_check[i]
                        left_half = re.search('([^=]*)', date)
                        right_half = re.search('"([^"]*)', date)
                        update_dict = {left_half.group(0) : right_half.group(0).strip('"')}
                tags = []
                if (match_OCL):
                    update_content=''
                    position=match_OCL.end()

                else:
                    open_gr = match_O.group(1)
                    temp = re.compile(r'</?'+open_gr+'[^<>]*?>')
                    tags = [con for con in temp.finditer(content,position)]
                    k=0
                    c=0
                    position=match_O.end()
                    for i,tag1 in enumerate(tags):
                        if tag1.group(0)[0]=='<' and tag1.group(0)[1]!='/':
                            k=k+1
                        else:
                            c=c+1
                        if k==c:
                            position=tag1.end()
                            break
                            
                    update_content = content[match_O.end():position]  
                self.children.append(XMLNode(match_O.group(1),update_dict,update_content))
            elif match_CL:
                if match_CL.group(1)!=tag:
                    raise Exception("Error at : <{0}> tag closed with {1}".format(tag, match_CL.group()))
                else:
                    self.content = self.content[:match_CL.start()]
                position = match_CL.end()   
            start = re.compile(r'<').search(content,position)
            if start:
                position = start.start()
            else:
                break

        pass # IMPLEMENT THIS

def xml_node_test(xml_node):
    # Test snippet:
    test_val = xml_node('<?xml version="1.0" encoding="UTF-8"?>')
    test.equal(str(test_val), '') # This should be ignored.
    test.equal(test_val.children, [])

    #Simple tag:
    test_val = xml_node("<body>Lorem Ipsum Dolor Sit Amet</body>")[0]
    test.equal(test_val.tag, "body")
    test.equal(test_val.content, "Lorem Ipsum Dolor Sit Amet")
    test.equal(test_val.count_tags(), 1)
    
    # Whitespace and string handling:
    test_val = xml_node("<body>     Lorem Ipsum <b  >Dolor</b> Sit <i>   Amet </i> </body>  ")[0]
    test.equal(test_val.tag, "body")
    test.equal([str(c) for c in test_val.children], ['Lorem Ipsum', '<b>Dolor</b>', 'Sit', '<i>Amet</i>'])
    test.equal(test_val.count_tags(), 3)

    # Attribute handling:
    test_val = xml_node("<body class='A' attr = ' b' foo='bar'   />")[0]
    test.equal(test_val.tag, "body")
    test.equal(list(test_val.attributes.items()), [('class', 'A'), ('attr ', ' b'), ('foo', 'bar')])
    test.equal(test_val.count_tags(), 1)

    # Exception throwing:
    test.exception(lambda: xml_node("<note><to>You</from></note>"), Exception)

    # Full parser test:
    test_val = xml_node(test_snippet)[0]
    test.equal(str(test_val), """<note date="8/31/12"><to>Tove</to><from>Jani</from><heading type="Reminder" /><body><h1>Don't</h1><p>forget me this weekend!</p></body>Here's some trailing text.</note>""")
    test.equal(test_val[0].tag, "to")
    test.equal(test_val[1].tag, "from")
    test.equal(test_val[2].attributes, {'type': 'Reminder'})
    test.equal(test_val[3][1].content, "forget me this weekend!")
    test.equal(test_val.count_tags(), 7)

    # Leading and trailing text:
    test.equal(str(xml_node("a<b>c</b>a<b>c</b>a<b>c</b>a")), "a<b>c</b>a<b>c</b>a<b>c</b>a")
    
    # Numbers in tags:
    test.equal(str(xml_node("<h1>A</h1>")), "<h1>A</h1>")
    
    # Nested quotes
    test.equal(str(xml_node("""<a href="localhost/">test</a>""")),"""<a href="localhost/">test</a>""")
    test.equal(str(xml_node("""<a href='localhost/'>test</a>""")),"""<a href="localhost/">test</a>""")
    test.equal(str(xml_node("""<a href="local'host/">test</a>""")),"""<a href="local'host/">test</a>""")
    test.equal(str(xml_node("""<a href='local"host/'>test</a>""")),"""<a href="local"host/">test</a>""")
    
    # Course webpage test:
    test_val = xml_node(course_webpage)
    test.equal(test_val.count_tags(), 126) # This value may change.    

In [214]:
@test
def xml_node(inp):
    return XMLNode.fromString(inp)

KeyboardInterrupt: 

There is a lot that this function needs to do, which is best explained by an example.  We'll eventually parse the `test_snippet` above using the command:

```python
root = XMLNode.fromString(test_snippet)
```

This will create a root node with a blank tag, an empty dictionary for the attributes, and children populated by parsing the test_snippet; the `content` attribute here will contain the entire test snippet.

The `children` attribute is a list that contains XMLNode instances corresponding to `node` tags and `string` instances corresponding to raw text between and inside tags.  Each time you instantiate an XMLNode, it must recursively create all its children. The call above would populate its children with:

```python
root.children = [XMLNode("note", {"date":"8/31/12"}, "\n    <to>Tove</to>\n    <from>Jani</from>...")]
```

This child node will have the tag `note` and attributes `{"date":"8/31/12"}`, plus content between `<note ...>` and `</note>`.  It, in turn, will have four children:

```python
root[0].children = [
    XMLNode("to", {}, "Tove"),
    XMLNode("from", {}, "Jani"),
    XMLNode("heading", {"type": "Reminder"}, "Tove"),
    XMLNode("body", {}, "Don't forget me this weekend!"),
]
```

Children of nodes can be XMLNodes or raw text, for example:

```python
root = XMLNode.fromString("Does this <i>  really</i> work?")
root.children = [
    "Does this",
    XMLNode("i", {}, "Really"),
    "work?"
]
```

Additionally, if you pass an XML object that is poorly formated, in that there is some mismatched open and close tag, the function should raise an exception.

```python
root = XMLNode("", {}, "<note><to>You</from></note>") # Throws exception
```

Make sure your code passes all the test cases!

Lets discuss in a bit more detail how the XML parsing will work algorithmically.  You being the initializer by copying the provided parameters to the class attributes.  Note that if you want you could make a full string copy here, but we don't bother.  Now we begin parsing the file, which we do by repeating the following logic until termination:
1. Look for the next xml tag (or comment, etc), in the file.  This is best done by finding the next `'<'` character.  If you can't find any, return.
2. If it's an xml prolog, html declaration, or comment, ignore this portion, and continue parsing after the prolog, declaration, or comment (i.e., throw away whatever information is contained in these portions)
3. If it's an open tag, read its tag and attributes (you'll likely want to use a regular expression to parse the attributes as well, but we leave this up to you).  If it's just an open tag, then recursively create an XMLNode object initializer this tag and attributes, and the content that occurred after the open tag.  If it's an open/close tag, create a XMLNode the same as before but with empty content.
4. If it's a close tag, make sure that the close tag matches the tag originally provided to the current XMLNode constructor (otherwise, we have a situation where one tag is closed with a different tag), and raise an Exception if not.  If the tags do match, then truncate the content to contain only the content before the closed tag matched, and return.

Some hints:
1. Keep track of the current position where you are parsing the file, and make sure to properly increment this so you move past any tag that you have parsed.
2. Make use of the `match = regular_expression_obj.match(string, pos)` function, which looks for a match to the regular expression starting _exactly_ as position `pos` in `string`.  If this function returns `None`, then the regular expression did not match.  In the returned `match` object, `match.end()` contains the position where the match ended.

## Q3: Searching for tags

One of the nicer elements of the `BeautifulSoup` library is the ability to quickly search for tags that have certain attributes, without worrying about the specific structure of the model (i.e., how many levels deep the tag is, how many may exist in the document etc).  We're going to implement a similar function in our `SearchableXMLNode` class, specifically:

In [ ]:
class SearchableXMLNode(XMLNode):
    @staticmethod
    def fromString(s):
        return SearchableXMLNode("", {}, s)

    def __init__(self, *a, **kw):
        super().__init__(*a, **kw)
        
    def create_child(self, *a, **kw):
        return SearchableXMLNode(*a, **kw)
    
    def find(self, tag=None, **properties):
        pass # IMPLEMENT THIS

def searchable_xml_node_test(searchable_xml_node):
    normalize = lambda l: sorted(str(s) for s in l)
    # Simple test:
    test_val = searchable_xml_node("<body>   <b>  Lorem </b> Ipsum <b  >Dolor</b> Sit <i>   Amet </i> </body>  ")[0]
    # find without filters yields all nodes, without any bare text:
    test.equal(normalize(test_val.find()), ['<b>Dolor</b>', '<b>Lorem</b>', '<body><b>Lorem</b>Ipsum<b>Dolor</b>Sit<i>Amet</i></body>', '<i>Amet</i>'])
    # find with just a tag:
    test.equal(normalize(test_val.find("b")), ['<b>Dolor</b>', '<b>Lorem</b>'])
    test.equal(normalize(test_val.find("i")), ['<i>Amet</i>'])
    test.equal(normalize(test_val.find("a")), [])

    # Course webpage test:
    test_val = searchable_xml_node(course_webpage)
    # find all links
    test.equal(normalize(c[0] for c in test_val.find("a")), ['Assignments', 'Calendar', 'FAQ', 'Information', 'Lectures', 'Policies', 'Practical Data Science', 'Staff', 'beautiful-jekyll', 'gmanek@cs.cmu.edu'])
    # find only links in navigation bar
    test.equal(normalize(c[0] for c in test_val.find("a", **{ "class": "btn" })), ['Assignments', 'Calendar', 'FAQ', 'Information', 'Lectures', 'Policies', 'Staff'])
    # Extract some page metadata
    test.equal(len(test_val.find("meta", name="author")), 1)
    test.equal(test_val.find("meta", name="author")[0].attributes["content"], "Practical Data Science")

In [ ]:
@test
def searchable_xml_node(inp):
    return SearchableXMLNode.fromString(inp)

For those who haven't seen the `**kwargs` parameter before, this is just a way to pass a variable-length list of parameters to a Python function as function parameters.  For example, you could call `find` via

```python
root.find("link", rel="stylesheet") -> [SearchableXMLNode("link", {...}, "...")]
```

and in the `find` function, `kwargs` would be a dictionary equal to `{"rel":"stylesheet"}`.

This function should return a list of _all_ SearchableXMLNodes that are descendents (children, children of children, etc), of the node you call it on. Providing no filter should match all nodes.